# Data Sampling Approaches

In this notebook we experiment with vaious data sampling approaches to improve the model's performance.

Our analysis of the data showed that the dataset had imbalanced classes. The number of examples without patronising and condescending language (PCL) is much higher than the number of examples with PCL. Models trained on imbalanced datasets may learn biased prior probabilities.

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaModel, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
dataset = load_dataset('csv', data_files={'train': 'train_dev_data/train_set.csv', 'test': 'train_dev_data/dev_set.csv'})

# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

In [ ]:
# Load model
model = RobertaModel.from_pretrained('roberta-base', num_labels=len(set(dataset['train']['label'])))

## Approach 1: oversampling

Random oversampling: a random choice of minority instances are duplicated.

In [ ]:
# Modify the minority instance data
dataset['train']['label']


# Tokenize dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets.set_format('torch')

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_roberta')
tokenizer.save_pretrained('./fine_tuned_roberta')

## Approach 2: undersampling

Random undersampling: a random choice of majority instances are removed from the dataset.